In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc2


In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device,
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/910 [00:00<?, ?it/s]

epoch:   0 loss:0.890532 auc:0.3809
epoch:  20 loss:0.286389 auc:0.9805
epoch:  40 loss:0.207907 auc:0.9814
epoch:  60 loss:0.179200 auc:0.9814
epoch:  80 loss:0.168816 auc:0.9834
epoch: 100 loss:0.159614 auc:0.9863
epoch: 120 loss:0.150605 auc:0.9863
epoch: 140 loss:0.147434 auc:0.9863
epoch: 160 loss:0.145358 auc:0.9863
epoch: 180 loss:0.144060 auc:0.9863
epoch: 200 loss:0.142793 auc:0.9873
epoch: 220 loss:0.140988 auc:0.9873


  0%|          | 1/910 [00:05<1:23:02,  5.48s/it]

epoch: 240 loss:0.141031 auc:0.9707
Fit finished.
epoch:   0 loss:0.742916 auc:0.5959
epoch:  20 loss:0.256651 auc:0.9714
epoch:  40 loss:0.189450 auc:0.9731
epoch:  60 loss:0.174989 auc:0.9763
epoch:  80 loss:0.166762 auc:0.9739
epoch: 100 loss:0.157952 auc:0.9747
epoch: 120 loss:0.150243 auc:0.9747
epoch: 140 loss:0.147080 auc:0.9747
epoch: 160 loss:0.144848 auc:0.9747


  0%|          | 2/910 [00:09<1:08:21,  4.52s/it]

...

Fit finished.
epoch:   0 loss:0.589303 auc:0.6236
epoch:  20 loss:0.252000 auc:0.9909
epoch:  40 loss:0.185439 auc:0.9955
epoch:  60 loss:0.169908 auc:0.9955
epoch:  80 loss:0.158980 auc:0.9955
epoch: 100 loss:0.150035 auc:0.9977
epoch: 120 loss:0.146842 auc:0.9977
epoch: 140 loss:0.144480 auc:0.9977
epoch: 160 loss:0.142889 auc:0.9932


100%|█████████▉| 909/910 [7:05:02<00:08,  8.39s/it]

epoch: 180 loss:0.140800 auc:0.9887
Fit finished.
epoch:   0 loss:0.838027 auc:0.3195
epoch:  20 loss:0.283678 auc:0.9763
epoch:  40 loss:0.213976 auc:0.9941
epoch:  60 loss:0.180790 auc:1.0000
epoch:  80 loss:0.168512 auc:1.0000
epoch: 100 loss:0.157620 auc:1.0000
epoch: 120 loss:0.149817 auc:1.0000
epoch: 140 loss:0.147086 auc:1.0000
epoch: 160 loss:0.145076 auc:1.0000
epoch: 180 loss:0.143743 auc:1.0000
epoch: 200 loss:0.142123 auc:1.0000
epoch: 220 loss:0.140528 auc:1.0000
epoch: 240 loss:0.138898 auc:1.0000
epoch: 260 loss:0.137008 auc:1.0000
epoch: 280 loss:0.136257 auc:1.0000
epoch: 300 loss:0.134628 auc:1.0000
epoch: 320 loss:0.134524 auc:1.0000
epoch: 340 loss:0.132480 auc:1.0000
epoch: 360 loss:0.130270 auc:1.0000
epoch: 380 loss:0.129698 auc:1.0000
epoch: 400 loss:0.127505 auc:1.0000
epoch: 420 loss:0.135042 auc:1.0000
epoch: 440 loss:0.125218 auc:1.0000
epoch: 460 loss:0.122986 auc:1.0000
epoch: 480 loss:0.121311 auc:1.0000
epoch: 500 loss:0.121276 auc:1.0000
epoch: 520 los

100%|██████████| 910/910 [7:05:20<00:00, 28.04s/it]

Fit finished.


In [7]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")